# Definitions
I define a child class for my simlutaion. It has a registry field to stores all the qubits necessary to run the simulation. 

In [31]:
from qat.lang.AQASM import Program, QRoutine, X, H, PH, RX, RY, RZ, SWAP
from qat.lang.AQASM.qint import QInt
import qat.lang.AQASM.classarith as classarith
import mpmath as mp
from qat.qpus import PyLinalg

# Quantum Fourier transform subroutine
def qFt(qbits):
    rout = QRoutine()
    wires = rout.new_wires(qbits)  
    for j in range(qbits):
        rout.apply(H, wires[j])
        for k in range(1, qbits-j):
            rout.apply(PH(-float(mp.pi)/2**k).ctrl(), wires[k+j], wires[j])
    if(qbits>1):
        for h in range(qbits//2):
            rout.apply(SWAP, wires[h], wires[qbits-h-1])
    return rout
# Inverse transform subroutine
def qFtInv(qbits):
    rout = QRoutine()
    wires = rout.new_wires(qbits)  
    for j in range(qbits):
        rout.apply(H, wires[j])
        for k in range(1, qbits-j):
            rout.apply(PH(float(mp.pi)/2**k).ctrl(), wires[k+j], wires[j])
    if(qbits>1):
        for h in range(qbits//2):
            rout.apply(SWAP, wires[h], wires[qbits-h-1])
    return rout
    
# Upon initialisation all the necessary qubits are allocated and all the simulation parameters are stored
class JLPsim(Program):
    def __init__(self, length, dimension, qbits, mass, spacing, wavefunction, cutoff):
        super().__init__()
# Store parameters
        self.length = length
        self.dim = dimension
        self.bits = qbits
        self.mass = mass
        self.spac = spacing
        self.cutoff = cutoff
# Create a cubic matrix with one register per lattice site. If dimension=2 the "cubic" matrix has only a single square slice.
        self.lattice = []
        for i in range(self.length):
            qmatrix = []
            if i==0 or self.dim==3:
                for j in range(self.length):
                    qrow = []
                    for k in range(self.length):
                        qrow.append(self.qalloc(self.bits, QInt))
                    qmatrix.append(qrow)
                self.lattice.append(qmatrix)
# Store the wavefunction flattened in lexicographic order
        self.wvfunc = []
        # for i in range(len(self.lattice)):
        #     for j in range(self.length):
        #         for k in range(self.length):
        #             self.wvfunc.append(wavefunction[i][j][k])
# Allocate ancillas
        # self.prep_ancilla = self.qalloc(1, QInt)
        # self.time_ancilla = self.qalloc(self.prec, QInt)
        self.estm_ancilla = self.qalloc(self.bits)
# Dispersion relation
    def energy(self, i, j, k):
        grad = mp.sin(i*mp.pi/self.length)**2 + mp.sin(j*mp.pi/self.length)**2 + mp.sin(k*mp.pi/self.length)**2
        return mp.sqrt(self.mass**2 + 4*grad/self.spac**2)
# Create vacuum matrix
    def build_VacuumMatrix(self):
        self.VacuumMatrix = []  
        for x1 in range(len(self.lattice)):
            for x2 in range(self.length):
                for x3 in range(self.length):
                    vrow = []
                    for y1 in range(len(self.lattice)):
                        for y2 in range(self.length):
                            for y3 in range(self.length):
                                Sigma = 0
                                for p1 in range(len(self.lattice)):
                                    for p2 in range(self.length):
                                        for p3 in range(self.length):
                                            Sigma += mp.cos(2*mp.pi*(p1*(x1-y1) + p2*(x2-y2) + p3*(x3-y3))/self.length) * energy(p1,p2,p3,)
                                vrow.append( (self.spac/self.length)**self.dim * Sigma )
                    self.VacuumMatrix.append(vrow)
# Calculate LDL decomposition
    def computeLDL(self):
        self.CholeskyDiagonal = []
        CholeskyTriangular = mp.eye(self.length**self.dim)
        for i in range(self.length**self.dim):
            diag = self.VacuumMatrix[i][i]
            for k in range(i):
                diag -= CholeskyTriangular[k,i]**2 * self.CholeskyDiagonal[k]
            self.CholeskyDiagonal.append(diag)
            for j in range(i+1,self.length**self.dim):
                CholeskyTriangular[i,j] = self.VacuumMatrix[i][j]
                for k in range(i):
                    CholeskyTriangular[i,j] -= CholeskyTriangular[k,i]*CholeskyTriangular[k,j]*self.CholeskyDiagonal[k]
                CholeskyTriangular[i,j] = CholeskyTriangular[i,j]/self.CholeskyDiagonal[i]
        self.InverseTriangular = CholeskyTriangular**-1
# Calculate correlation function
    def build_CorrelationFunction(self):
        self.CorrelationFunction = (1/2)*mp.matrix(self.VacuumMatrix)**-1
# Calculate normalisation factor
    def compute_normalisation(self):
        vec = self.CorrelationFunction*mp.matrix(self.wvfunc)
        eta = 0
        for i in range(self.length**self.dim):
            eta += mp.conj(self.wvfunc[i])*vec[i]
        self.norm = 1/mp.sqrt(mp.re(eta))
# Phi-sector wavepacket preparation unitary
    def FieldWaveUnitary(self, labels, time):
        gamma = 0
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    gamma += self.wvfunc[i*self.length**2+j*self.length+k] * labels[i][j][k]
                    for l in range(self.bits):
                        if (labels[i][j][k]//2**(self.bits-l-1))%2 == 0:                   
                            self.apply(X, self.lattice[i][j][k][l])
        gamma *= self.norm/2
        self.apply(RZ(float(mp.arg(gamma))).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        self.apply(RX(2*float(mp.fabs(gamma))*time).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        self.apply(RZ(float(-mp.arg(gamma))).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    for l in range(self.bits):
                        if (labels[i][j][k]//2**(self.bits-l-1))%2 == 0:
                            self.apply(X, self.lattice[i][j][k][l])
# Phi-sector wavepacket preparation overall subroutine
    def FieldWaveEvolution(self, time):
        for n in range(2**(self.bits*self.length**self.dim)):
            labels = []
            for i in range(self.length):
                intmatrix = []
                if i==0 or self.dim==3:
                    for j in range(self.length):
                        introw = []
                        for k in range(self.length):
                            introw.append((n//2**(self.bits*(self.length**self.dim-3*i-2*j-k-1)))%2**self.bits)
                        intmatrix.append(introw)
                    labels.append(intmatrix)
            FieldWaveUnitary(labels, time)
# Pi-sector wavepacket preparation unitary
    def ConjugateWaveUnitary(self, labels, time):
        tau = 0
        vec = self.CorrelationFunction*mp.matrix(self.wvfunc)
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    tau += self.spac**self.dim * vec[i*self.length**2+j*self.length+k] * labels[i][j][k]
                    for l in range(self.bits):
                        if (labels[i][j][k]//2**(self.bits-l-1))%2 == 0:
                            self.apply(X, self.lattice[i][j][k][l])
        tau *= self.norm/mp.j
        self.apply(RZ(float(mp.arg(tau))).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        self.apply(RX(2*float(mp.fabs(tau))*time).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        self.apply(RZ(float(-mp.arg(tau))).ctrl(self.bits*self.length**self.dim), self.lattice, self.prep_ancilla)
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    for l in range(self.bits):
                        if (labels[i][j][k]//2**(self.bits-l-1))%2 == 0:
                            self.apply(X, self.lattice[i][j][k][l])
# Pi-sector wavepacket preparation overall subroutine
    def ConjugateWaveEvolution(self, time):
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    self.apply(qFt(self.bits), self.lattice[i][j][k])
        for n in range(2**(self.bits*self.length**self.dim)):
            labels = []
            for i in range(self.length):
                intmatrix = []
                if i==0 or self.dim==3:
                    for j in range(self.length):
                        introw = []
                        for k in range(self.length):
                            introw.append((n//2**(self.bits*(self.length**self.dim-3*i-2*j-k-1)))%2**self.bits)
                        intmatrix.append(introw)
                    labels.append(intmatrix)
            ConjugateWaveUnitary(labels, time)
        for i in range(len(self.lattice)):
            for j in range(self.length):
                for k in range(self.length):
                    self.apply(qFtInv(self.bits), self.lattice[i][j][k])
##############
    def MomentumFieldPhase(self):
        dummy = self.qalloc(self.bits, QInt)
        for x1 in range(len(self.lattice)):
            for x2 in range(self.length):
                for x3 in range(self.length):
                    for y1 in range(len(self.lattice)):
                        for y2 in range(self.length):
                            for y3 in range(self.length):
                                dummy += self.lattice[x1][x2][x3]
                                self.time_ancilla += dummy*self.lattice[y1][y2][y3]
                                self.reset(dummy)
# Quatum phase estimation
    def measure_momentum(self):
        U = QRoutine()
        wires = U.new_wires(self.bits*self.length**self.dim)
        for j in range(self.bits*self.length**self.dim):
            U.apply(X, wires[j])
        for i in range(self.bits):
            self.apply(H, self.estm_ancilla[self.bits-i-1])
            self.apply(U.ctrl(), self.estm_ancilla[self.bits-i-1], self.lattice)
        self.apply(qFt(self.bits), self.estm_ancilla)
        

simu = JLPsim(3,2,2,1,1,[],0)
simu.measure_momentum()

qpu = PyLinalg()
circ = simu.to_circ()
job = circ.to_job(qubits=simu.estm_ancilla)
result = qpu.submit(job)
for sample in result:
    print("State {} Amplitude: {}".format(sample.state,sample.amplitude))

State |00> Amplitude: None
State |01> Amplitude: None
State |11> Amplitude: None


# Testing

In [26]:
simu = Program()
C = AbstractGate("C", [], arity=1)
x = simu.qalloc(1)
simu.apply(C, x)
circ = simu.to_circ
%qatdisplay circ

TypeError: __class__ assignment only supported for heap types or ModuleType subclasses